In [10]:
# Importar las bibliotecas necesarias
import os
import sqlite3
import pandas as pd
import numpy as np
import regex as re
import seaborn as sns
from tqdm import tqdm


ruta = "C:/Users/jalarcon/OneDrive - GRUPO ASESORIA EN SISTEMATIZACIÓN DE DATOS SAS/Escritorio/Proyecto_Aplicado/Lotes"

# Lista de archivos en la carpeta
files = os.listdir(ruta)
                                                                                                                                                                                                                                                                        
# DataFrame vacío para almacenar los datos
df = pd.DataFrame()

# Lista de columnas que se esperan en los archivos Excel
columnas = [
    'ImportationId', 'Apellidos', 'CapitalIcetexAct', 'IdSolicitante', 'IDSolicitud',
    'Nombres', 'NUMDIASMORA_ACT', 'SALDO TOTAL', 'SALDO MORA', 'FECHAINICIO',
    'FECHA_FIN', 'CAMPAÑA', 'MOTOR_MARCACION', 'MODO_MARCACION',
    'CALIFICACION_MARCADOR', 'TELEFONO', 'ID_Llamada', 'AGENTE', 'HOLD', 'ABANDONED'
]

# Leer cada archivo de Excel en la carpeta
for Lotes in files:
    try:
        # Intentar leer el archivo de Excel
        df_lotes = pd.read_excel(os.path.join(ruta, Lotes), header=1)
        
        # Concatenar el DataFrame actual con el nuevo DataFrame leído
        df = pd.concat([df, df_lotes])

    except Exception as e:
        # Manejar errores al leer archivos y mostrar mensajes de error
        print(f"Error al leer el archivo {Lotes}: {e}")

# Eliminar duplicados por fila en el DataFrame resultante
df.reset_index(drop=True, inplace=True)

# Obtener los primeros 10 datos del DataFrame
primeros_1000_datos = df.head(10)

# Imprimir los primeros 10 datos
print(primeros_1000_datos)

# Función para convertir valores a enteros sin decimales
def entero(x):
    try:
        # Utilizar la función int() directamente para asegurar que no haya decimales
        return int(x)
    except:
        return np.nan

# Muestra las columnas y las primeras 8 filas del DataFrame
df.columns
df.head(8)

# Elimina la columna "IDSolicitud"
df = df.drop(columns=["IDSolicitud"])

# Muestra las primeras 10 filas del DataFrame después de la eliminación
df.head(10)

# Información sobre el DataFrame
df.info()

# Lista de columnas que se van a eliminar
columnas_a_eliminar = [
    'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 7',
    'Unnamed: 8', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 13',
    'Unnamed: 14', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 19',
    'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 23', 'Unnamed: 24',
    'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 29', 'Unnamed: 30',
    'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 36',
    'Unnamed: 37', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 42',
    'Unnamed: 43', 'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47',
    'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 53',
    'Unnamed: 54', 'Unnamed: 56', 'Unnamed: 57', 'Unnamed: 59'
]

# Elimina las columnas no deseadas
df = df.drop(columnas_a_eliminar, axis=1, errors='ignore')

# Elimina duplicados por fecha de inicio y ID de solicitante
df.drop_duplicates(subset=["FECHA_INICIO", "IdSolicitante"], inplace=True)

# Agrega una nueva columna "LIMPIEZA_CEL" que contiene dígitos del número de teléfono
df["LIMPIEZA_CEL"] = df["TELEFONO"].apply(lambda x: "".join(re.findall("\d+", x)[1:]))

# Convierte las columnas de fecha a formato datetime
df["FECHA_INICIO"] = pd.to_datetime(df["FECHA_INICIO"], yearfirst=True)
df["FECHA_FIN"] = pd.to_datetime(df["FECHA_FIN"], yearfirst=True, format="mixed")

# Convierte las columnas "IdSolicitud" e "IdSolicitante" a enteros sin decimales
df["IdSolicitud"] = df["IdSolicitud"].apply(lambda x: entero(x))
df["IdSolicitante"] = df["IdSolicitante"].apply(lambda x: entero(x))

# Información sobre el DataFrame después de las transformaciones
df.info()

# Importar bibliotecas para cifrado
from cryptography.fernet import Fernet

# Generar una clave de cifrado
key = Fernet.generate_key()
cipher_suite = Fernet(key)

# Cifrar los datos en la columna "Nombres"
df['columna_ofuscada'] = df['Nombres'].apply(lambda x: cipher_suite.encrypt(str(x).encode()).decode() if not pd.isnull(x) else None)

# Mostrar las primeras filas después de agregar la columna cifrada
df.head()

# Eliminar la columna original "Nombres" si se desea ocultar
df.drop('Nombres', axis=1, inplace=True)

# Mostrar las columnas del DataFrame
df.columns

# Conectar a la base de datos SQLite y guardar el DataFrame en una tabla llamada "Gestion_lotes"
con = sqlite3.connect("Lotes.db")
df.to_sql("Gestion_lotes", con, if_exists="replace", index=False)

# Guardar el DataFrame como un archivo CSV
df.to_csv('C:/Users/jalarcon/OneDrive - GRUPO ASESORIA EN SISTEMATIZACIÓN DE DATOS SAS/Escritorio/Proyecto_Aplicado/Allegro_Lotes.csv', index=False, sep=";", encoding='iso-8859-1')


Error al leer el archivo C_IN_09062023_20_62_Reporte.xls: 
Error al leer el archivo C_IN_16072023_30_80_I_Reporte.xls: 253
Error al leer el archivo C_IN_22092023_15_80_II_Reporte.xls: list index out of range
Error al leer el archivo C_IN_28072023_31_82_1_Reporte.xls: list index out of range
        ImportationId  Apellidos  CapitalIcetexAct  IdSolicitante   
0  C_IN_1062023_19_34        NaN       13915080.29   1.043024e+09  \
1  C_IN_1062023_19_34        NaN       33250237.95   1.043660e+09   
2  C_IN_1062023_19_34        NaN        9931929.53   1.043664e+09   
3  C_IN_1062023_19_34        NaN        2695037.99   1.043124e+09   
4  C_IN_1062023_19_34        NaN       18572832.63   1.043124e+09   
5  C_IN_1062023_19_34        NaN         281578.44   1.043295e+09   
6  C_IN_1062023_19_34        NaN       10365653.52   1.043441e+09   
7  C_IN_1062023_19_34        NaN        6304188.81   1.043114e+09   
8  C_IN_1062023_19_34        NaN        8073460.26   1.043024e+09   
9  C_IN_1062023_19

In [9]:
df.head(5)

AttributeError: 'list' object has no attribute 'head'

In [8]:
df_query = ['TELEFONO','LIMPIEZA_CEL','columna_ofuscada','NUMDIASMORA_ACT']